In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [2]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

$V_i = ASC + F(GC(T, C), b_{gc_i})$

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [3]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [4]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
         'mode_model', 'purpose_vp', 'car_avail', 'length']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [5]:
inf = 10000
df = df.replace({np.inf:inf})

In [6]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,...,T_NON_MOTOR,T_AIR,C_RAIL,C_ROAD,T_RAIL,T_ROAD,MODE,PURPOSE,CAR_AV,DIST
count,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.0,94638.000000,94638.000000,94638.000000,94638.000000,...,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000,94638.000000
mean,19.523429,87.365557,8.367430,7.580687,3.250919,0.0,9556.283078,114.355315,5246.966102,80.368870,...,4095.421490,9567.313110,18.190574,4.929169,90.275287,1156.758833,5.549515,3.652476,0.965236,95.581019
std,25.139820,54.788528,12.013638,6.741391,1.270667,0.0,2041.829333,348.050585,4923.958941,85.139514,...,4866.585903,1991.159719,18.984434,5.221736,91.087131,3042.673831,1.350781,2.059686,0.187290,135.668942
min,0.000000,19.000000,0.010944,5.000000,0.000000,0.0,52.277023,17.433333,21.033333,10.400000,...,6.683333,5.425000,0.000000,0.000000,17.433333,18.200000,1.000000,1.000000,0.000000,0.265000
25%,6.188580,24.481557,2.349052,5.000000,2.740000,0.0,10000.000000,35.283333,120.533333,36.350000,...,76.950000,10000.000000,7.185460,3.600000,35.283333,45.533333,6.000000,1.000000,1.000000,29.503000
50%,10.934091,139.000000,4.480332,5.000000,4.000000,0.0,10000.000000,50.950000,10000.000000,49.816667,...,120.816667,10000.000000,12.062743,4.000000,50.150000,73.450000,6.000000,3.000000,1.000000,45.032000
75%,17.718818,139.000000,8.436239,5.000000,4.000000,0.0,10000.000000,104.483333,10000.000000,75.183333,...,10000.000000,10000.000000,20.994973,4.000000,100.816667,180.733333,6.000000,6.000000,1.000000,79.838000
max,100.000000,139.000000,112.545020,55.777715,4.000000,0.0,10000.000000,10000.000000,10000.000000,651.633333,...,10000.000000,10000.000000,139.000000,48.821179,647.575000,10000.000000,7.000000,6.000000,9.000000,985.472000


In [7]:
# Drop non-motorised trips over 40km, because there are erronous OD pairs
#df = df.loc[~((df['MODE']==7)&(df['DIST']>40))]

In [8]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [9]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [10]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [11]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})

In [12]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
drop_ratio = len(df.loc[mask])/len(df.loc[df['MODE']==6])
lengths = list(df.loc[mask, 'DIST'])
print('Share of car trips dropped: {}'.format(drop_ratio))
df = df.loc[~mask]
# Drop other modes equally
#for m in [1,4]:
#    n_drops = int(np.round(drop_ratio * len(df.loc[df['MODE']==m])))
#    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
#                        (df['DIST']>min(lengths))].sample(n_drops).index)
#    print('Dropped ' + str(n_drops) + ' trips of mode ' + str(m))
print(len(df))

Share of car trips dropped: 0.0141490154593321
93461


In [13]:
# Check that PT trips have no infinity cost
for mode, col in zip([1, 4, 5], ['RAIL_AV', 'ROAD_AV', 'AIR_AV']):
    assert len(df.loc[((df['MODE']==mode) & ~(df[col]))].index) == 0, str(mode)

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [14]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [15]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
663,20.1531,19.2827,9.18,11.9,26.2236,15.2133,15.1837,11.6458,13.48,16.0758,...,68.5735,68.5735,160.8889,38.7422,19.9379,18.9249,11.9147,25.2417,22.0095,15.2133
405,16.5920,15.9310,9.18,11.9,19.1460,12.8480,12.6110,9.9310,13.48,14.3610,...,38.8170,38.8170,46.0720,23.2860,16.4810,15.7280,10.1210,21.1090,17.7190,12.8480


In [16]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [17]:
df = df.loc[df['DIST']<=1000]
len(df)

93461

In [18]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_Sc'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                   for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_ROAD'] = df['T_ROAD_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_ROAD'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [19]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

93461

### Estimation parameters

In [20]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 0)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [21]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [22]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [23]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [24]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [25]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [26]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)

In [27]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL)
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_ROAD = asc_road + spline(GC_ROAD)
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [28]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [29]:
# Map modes to utility functions
V = {1:V_RAIL,
#     1:V_RAIL_S,
#     2:V_RAIL_L,
#     3:V_COACH,
     4:V_ROAD,#BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [30]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_AV,#RAIL_S_AV,
#      2:RAIL_L_AV,
#      3:COACH_AV,
      4:ROAD_AV,#BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [31]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [32]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results_car_own.xls', engine='xlsxwriter')

In [33]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [34]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [35]:
results_mnl = model_mnl.estimate()

In [36]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-4.680171,0.164173,-28.5076,0,0.178006,-26.2922,0
asc_non_motor,-4.564615,0.0420092,-108.658,0,0.0422174,-108.122,0
asc_rail,-2.311486,0.0148908,-155.229,0,0.020028,-115.413,0
asc_road,-3.159994,0.0185346,-170.492,0,0.0196193,-161.066,0
b_gc,-0.218560,0.00705523,-30.9785,0,0.0154436,-14.1522,0
Number of estimated parameters,5.000000,,,,,,
Sample size,93461.000000,,,,,,
Excluded observations,0.000000,,,,,,
Init log likelihood,-127151.560217,.7g,,,,,
Final log likelihood,-37988.554365,.7g,,,,,


In [37]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [38]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [39]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [40]:
results_nl = model_nl.estimate()

In [41]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-4.683667,0,0.167201,-28.0122,0,0.190864,-24.5393,0
asc_non_motor,-4.564583,0,0.0420235,-108.62,0,0.0422933,-107.927,0
asc_rail,-2.311516,0,0.0273893,-84.3948,0,0.0676845,-34.1514,0
asc_road,-3.160017,0,0.0954889,-33.093,0,0.258325,-12.2327,0
b_gc,-0.218585,0,0.01283,-17.037,0,0.0308028,-7.09627,1.28164e-12
mu_pt,1.000000,1,0.0915677,10.9209,0,0.251049,3.98329,6.79679e-05
Number of estimated parameters,6.000000,,,,,,,
Sample size,93461.000000,,,,,,,
Excluded observations,0.000000,,,,,,,
Init log likelihood,-127151.560217,.7g,,,,,,


In [42]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [43]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    for car_ownership in [0,1]:
        database = db.Database('MiD2017', df.copy())
        database.remove(PURPOSE!=p)
        if p!=4: # Estimate business trips together because of small sample size
            database.remove(CAR_AV!=car_ownership)
        print('Sample size for purpose {}, car_ownership {}: {}'.format(
            p, car_ownership, database.getSampleSize()))
        model = bio.BIOGEME(database, mnl) # Choose the model formulation
        model.modelName = 'MNL_Fz' + str(p) + '_' + str(car_ownership) # Name it
        results.append(model.estimate()) # Estimation
        output = results[-1].getEstimatedParameters()
        # Add results to the Excel file
        for key, val in results[-1].getGeneralStatistics().items():
            output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
        output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1, car_ownership 0: 644
Sample size for purpose 1, car_ownership 1: 24620
Sample size for purpose 2, car_ownership 0: 117
Sample size for purpose 2, car_ownership 1: 3464
Sample size for purpose 3, car_ownership 0: 400
Sample size for purpose 3, car_ownership 1: 22519
Sample size for purpose 4, car_ownership 0: 5611
Sample size for purpose 4, car_ownership 1: 5611
Sample size for purpose 6, car_ownership 0: 872
Sample size for purpose 6, car_ownership 1: 35214


In [44]:
writer.save()